In [1]:
import pandas as pd
import numpy as np
import csv
import os
import sys
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stat
import math

In [2]:
combined_maf = pd.read_csv('/Users/lmartin/Documents/2025/RA_combined_maf_dedup_updated_on_trees_022025.maf',sep='\t')
subclones_sig_genes = pd.read_csv('/Users/lmartin/Documents/2023/CDK4_mutsig_added_pats/on_subclones_v3/0/mutsig_results/outdir/sig_genes.txt',sep='\t')



In [3]:
subset=(subclones_sig_genes['p'] < 0.001)

In [4]:
coding_muts=['Missense_Mutation',
 'Nonsense_Mutation',
 'Start_Codon_SNP',
 'De_novo_Start_InFrame',
 'De_novo_Start_OutOfFrame',
 'Nonstop_Mutation',
 'Frame_Shift_Del',
 'Frame_Shift_Ins',
 'In_Frame_Del',
 'In_Frame_Ins']

coding_maf=combined_maf[combined_maf['Variant_Classification'].isin(coding_muts)]
len(coding_maf['Hugo_Symbol'].unique())

6790

In [5]:
pd.concat([
  coding_maf.groupby(["Hugo_Symbol", "Patient_ID"]).size().loc[subclones_sig_genes.loc[subset, "gene"]].groupby(level = 0).size().rename("n_mut_pats"),
  coding_maf.groupby(["Hugo_Symbol", "Tumor_Sample_Barcode"]).size().loc[subclones_sig_genes.loc[subset, "gene"]].groupby(level = 0).size().rename("n_mut_clones")],
  axis = 1
)

,n_mut_pats,n_mut_clones
Hugo_Symbol,,
CSAD,2,3
CTCF,2,4
CTNNA2,5,5
ESR1,6,10
HIST1H3B,2,3
IVL,4,6
KMT2C,5,16
KRAS,3,3
MEN1,1,2


## test with KMT2C

In [6]:
KMT2C_all_muts = coding_maf[coding_maf['Hugo_Symbol']=='KMT2C']
KMT2C_all_muts

,Patient_ID,Sample_ID,Sample_Alias,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele,t_ref_count,t_alt_count,...,Cluster_Assignment,Allelic_CN_minor,Allelic_CN_major,preDP_ccf_mean,preDP_ccf_CI_low,preDP_ccf_CI_high,clust_ccf_mean,clust_ccf_CI_low,clust_ccf_CI_high,Tumor_Sample_Barcode
731,1547,6,NaN,KMT2C,7,151874772,C,T,180,6,...,15,NaN,NaN,0.09,0.04,0.14,0.05,0.05,0.06,1547_CL15
854,1547,10,NaN,KMT2C,7,151879260,C,A,184,12,...,14,NaN,NaN,0.18,0.10,0.27,0.06,0.06,0.06,1547_CL14
3714,1558,RA_1558_cfDNA_3,NaN,KMT2C,7,151879322,G,A,79,33,...,1,NaN,NaN,0.89,0.76,1.00,1.00,0.99,1.00,1558_CL1
3957,1558,1558_6,NaN,KMT2C,7,152012253,G,C,185,21,...,10,NaN,NaN,0.39,0.25,0.53,0.30,0.29,0.30,1558_CL10
4071,1326,RP-1700_BWES00001_v1_Exome_OnPrem,NaN,KMT2C,7,151896451,T,A,36,13,...,2,NaN,NaN,0.80,0.57,1.00,0.98,0.96,1.00,1326_CL2
4208,1326,RP-1700_T01027-3_v4_Exome_OnPrem,NaN,KMT2C,7,151945382,C,-,246,64,...,9,NaN,NaN,0.87,0.74,1.00,0.83,0.77,0.89,1326_CL9
4701,2389,1,NaN,KMT2C,7,151891593,G,A,101,46,...,5,NaN,NaN,0.77,0.62,0.92,0.88,0.86,0.90,2389_CL5
5037,2389,3,NaN,KMT2C,7,151859429,C,A,185,4,...,18,NaN,NaN,0.08,0.03,0.12,0.01,0.01,0.01,2389_CL18
5525,2389,4,NaN,KMT2C,7,151851222,G,C,158,10,...,4,NaN,NaN,0.22,0.11,0.36,0.16,0.15,0.17,2389_CL4
5526,2389,4,NaN,KMT2C,7,151878626,C,A,122,5,...,12,NaN,NaN,0.16,0.06,0.25,0.36,0.35,0.37,2389_CL12


In [7]:
## total number of mutations in the target gene N0
N0 = len(KMT2C_all_muts)
## total number of clones
N_c = len( coding_maf['Tumor_Sample_Barcode'].unique() )

In [8]:
N0

21

In [9]:
N_c

159

## start with one patient

In [10]:
RA_2389 = coding_maf[coding_maf['Patient_ID']==2389]

### for every clone, I am going to model the probability that KMT2C is in the clone or not
### then I'll tell it what the real distribution is and the sum is the score

In [58]:

RA_2389_clones = list(RA_2389['Tumor_Sample_Barcode'].unique() )
# total number of mutations in this patient
M = len(RA_2389)
 
# initialize a Pij matrix where you have a 1 if a clone was mutated and then the score
P_2389 =np.zeros( (len(RA_2389_clones), 2 ))

gene="KMT2C"

clone_idx=0
for clone in RA_2389_clones:
    clone_df = RA_2389[RA_2389['Tumor_Sample_Barcode'] == clone]
    mutations_in_clone = clone_df['Hugo_Symbol'].unique()
    Mij = len(list(mutations_in_clone))
    
    term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
    
    # a given clone has no mutation in the gene
    P_2389[clone_idx, 0] =  term_to_calc
    
    # a given clone has one or more mutations in the gene
    P_2389[clone_idx, 1] = 1- term_to_calc
    

    clone_idx +=1


In [59]:
## now every row sums to one

In [60]:
P_2389_prob_df = pd.DataFrame(P_2389, columns=['no_KMT2C', 'KMT2C'])
P_2389_prob_df.insert(0, 'clone', RA_2389_clones)
P_2389_prob_df

,clone,no_KMT2C,KMT2C
0,2389_CL16,0.862730,0.137270
1,2389_CL5,0.938615,0.061385
2,2389_CL3,0.927032,0.072968
3,2389_CL18,0.359055,0.640945
4,2389_CL14,0.916851,0.083149
5,2389_CL15,0.758178,0.241822
6,2389_CL1,0.979575,0.020425
7,2389_CL17,0.834506,0.165494
8,2389_CL6,0.920656,0.079344
9,2389_CL7,0.939910,0.060090


In [64]:
RA_2389.groupby('Tumor_Sample_Barcode').count()

,Patient_ID,Sample_ID,Sample_Alias,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele,t_ref_count,t_alt_count,...,Variant_Type,Cluster_Assignment,Allelic_CN_minor,Allelic_CN_major,preDP_ccf_mean,preDP_ccf_CI_low,preDP_ccf_CI_high,clust_ccf_mean,clust_ccf_CI_low,clust_ccf_CI_high
Tumor_Sample_Barcode,,,,,,,,,,,,,,,,,,,,,
2389_CL1,17,17,0,17,17,17,17,17,17,17,...,17,17,0,0,17,17,17,17,17,17
2389_CL10,22,22,0,22,22,22,22,22,22,22,...,22,22,0,0,22,22,22,22,22,22
2389_CL11,65,65,0,65,65,65,65,65,65,65,...,65,65,0,0,65,65,65,65,65,65
2389_CL12,87,87,0,87,87,87,87,87,87,87,...,87,87,0,0,87,87,87,87,87,87
2389_CL13,206,206,0,206,206,206,206,206,206,206,...,206,206,0,0,206,206,206,206,206,206
2389_CL14,64,64,0,64,64,64,64,64,64,64,...,64,64,0,0,64,64,64,64,64,64
2389_CL15,205,205,0,205,205,205,205,205,205,205,...,205,205,0,0,205,205,205,205,205,205
2389_CL16,112,112,0,112,112,112,112,112,112,112,...,112,112,0,0,112,112,112,112,112,112
2389_CL17,133,133,0,133,133,133,133,133,133,133,...,133,133,0,0,133,133,133,133,133,133


## patient score if no KMT2C mutation in any clone

In [13]:
P_2389_prob_df_no_KMT2C = P_2389_prob_df['no_KMT2C'].to_list()
logs_no_KMT2C = [ np.log(p) for p in P_2389_prob_df_no_KMT2C]
print( -np.sum(logs_no_KMT2C))

2.6946982684320844


## patient score if a KMT2C mutation in all clones

In [14]:
P_2389_prob_df_KMT2C = P_2389_prob_df['KMT2C'].to_list()
logs_KMT2C = [ np.log(p) for p in P_2389_prob_df_KMT2C]
print( -np.sum(logs_KMT2C))

37.92316367280446


In [ ]:
def compute_score_real():
    probs_pats_KMT2C_scores = []

    all_clones_in_pat = list(pat_probs['clone'])
    for clone in all_clones_in_pat:
        pat_probs_clone = pat_probs[pat_probs['clone']==clone]
        if clone in all_KMT2C_clones:

            pat_probs_clone_KMT2C = pat_probs_clone['KMT2C'].to_list()
            logs_KMT2C = [ - np.log(p) for p in pat_probs_clone_KMT2C]
            running_score.extend(logs_KMT2C)
        else:
            pat_probs_no_KMT2C = pat_probs_clone['no_KMT2C'].to_list()
            logs_no_KMT2C = [ - np.log(p) for p in pat_probs_no_KMT2C]
            running_score.extend(logs_no_KMT2C)
            
    score = np.sum(running_score)
        
    probs_pats_KMT2C_scores.append(score)


In [ ]:
def compute_score_perms():

    return

## permutations

In [ ]:
## so I am using the curveball to continue to permute the number of mutations per clone given the number of mutations and clones in the data
## again start on a per patient basis

## OK so my distribution is the probability matrix that I get from the observed
## then through the curveball, I permute # of clones mutated and calculate the score
## then compare the real score to the observed score under the perms

## in this case it is also important to know which clones I am permuting

In [ ]:
maf = coding_subclones_maf
clones, c_ui, c_idx = np.unique(maf["Tumor_Sample_Barcode"], return_inverse = True, return_index = True)

_, p_idx = np.unique(maf["Patient_ID"], return_inverse = True)
c2pat = p_idx[c_ui]

# genes
genes, g_ui, g_idx = np.unique(maf["Hugo_Symbol"], return_inverse = True, return_index = True)
g2gidx = dict(zip(genes, g_idx[g_ui]))

subset_idx_mat = [g2gidx[x] for x in S.loc[subset_idx, "gene"]]

# CG is matrix size # total of genes x total # of clones
CG = np.array((sp.coo_matrix((np.ones_like(c_idx), (c_idx, g_idx))).todense() > 0))


In [66]:
RA2389_obs_score = probs_pats_with_KMT2C_scores_dict[2389]

In [ ]:

# precompute nonzero elements
hp = curveball.find_presences(CG)

#
# run permutations (in parallel)
pool = multiprocessing.Pool()

n_perm = 10000
n_chunk = 1000

n_genes=1
n_iterations = n_perm // n_chunk 
total_clones = np.arange(1, len(RA_2389_clones) )
mutation_counts = np.zeros((n_genes, len(RA_2389_clones)))
test_matrix = np.zeros((n_iterations, n_genes))

test = np.zeros_like(null_pat_sum)
for p_chunk in range(n_perm//n_chunk):
    permute_futures = [None]*n_chunk
    for p in range(n_chunk):
        
        permute_futures[p] = pool.apply_async(curveball.curve_ball, (CG, hp, 1000) )
        
        #permute_futures[p].get() = size of total # of clones

    # compute permutation p-values
    for p in range(n_chunk):

        # comput_pat_sums(total # of clones(get cell of matrix[:, genes we are testing (14)] <= observed distribution)
        
        # returns size 14 matrix because 14 genes, each entry is the # of patients where that gene was mutated
        itr_test = ( compute_pat_sums(c2pat, permute_futures[p].get()[:, subset_idx_mat]))
        
        # c2pat maps clones to patients
        test += ( compute_pat_sums(c2pat, permute_futures[p].get()[:, subset_idx_mat]) <= RA2389_obs_score)

        for gene_idx, num_mut_patients in enumerate(itr_test):
            if 1 <= num_mut_patients <= 15:  
                mutation_counts[gene_idx, num_mut_patients - 1] += 1
    
    # tracking the final deviation from the observed distribution of # patients with a given mutation on each step         
    test_matrix[p_chunk, :] = itr_test.copy()
    
    print(f"Chunk {p_chunk + 1}/{n_perm//n_chunk} finished.")

#Cb = pd.DataFrame({ "gene" : genes[subset_idx_mat], "p" : test/(n_chunk*(n_perm//n_chunk)) })

Cb = pd.DataFrame({ "gene" : genes[subset_idx_mat], "p" : test/(n_chunk*(n_perm//n_chunk)),
                      "n_mut_clones": n_mut_clones[subset_idx_mat], 
                    "n_mut_pats": n_mut_pats[subset_idx_mat] })

## test RA 1598, no KMT2C

In [15]:
RA_1598= coding_maf[coding_maf['Patient_ID']==1598]
RA_1598_clones = list(RA_1598['Tumor_Sample_Barcode'].unique() )

# total number of mutations in this patient
M = len(RA_1598)
# initialize a Pij matrix where you have a 1 if a clone was mutated 
P_1598 =np.zeros( (len(RA_1598_clones), 2 ))

gene="KMT2C"

clone_idx=0
for clone in RA_1598_clones:
    clone_df = RA_1598[RA_1598['Tumor_Sample_Barcode'] == clone]
    mutations_in_clone = clone_df['Hugo_Symbol'].unique()
    Mij = len(list(mutations_in_clone))
    term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
    # a given clone has no mutation in the gene
    P_1598[clone_idx, 0] =  term_to_calc
    
    #  a given clone has a mutation in the gene
    P_1598[clone_idx, 1] = 1-term_to_calc
    

    clone_idx +=1

In [16]:
P_1598_prob_df = pd.DataFrame(P_1598, columns=['no_KMT2C', 'KMT2C'])
P_1598_prob_df.insert(0, 'clone', RA_1598_clones)
P_1598_prob_df

,clone,no_KMT2C,KMT2C
0,1598_CL2,0.931663,0.068337
1,1598_CL1,0.867789,0.132211
2,1598_CL8,0.700242,0.299758
3,1598_CL10,0.269250,0.730750
4,1598_CL5,0.883355,0.116645
5,1598_CL4,0.931663,0.068337
6,1598_CL7,0.793799,0.206201
7,1598_CL9,0.867789,0.132211
8,1598_CL3,0.899186,0.100814
9,1598_CL6,0.793799,0.206201


## patient score if no KMT2C clones (truth)

In [17]:
P_1598_prob_df_no_KMT2C = P_1598_prob_df['no_KMT2C'].to_list()
logs_no_KMT2C_1598 = [ np.log(p) for p in P_1598_prob_df_no_KMT2C]
print( -np.sum(logs_no_KMT2C_1598))

2.785767361195695


## patient score if all KMT2C clones

In [18]:
P_1598_prob_df_KMT2C = P_1598_prob_df['KMT2C'].to_list()
logs_KMT2C_1598 = [ np.log(p) for p in P_1598_prob_df_KMT2C]
print( -np.sum(logs_KMT2C_1598))

18.532696469275578


## try for all patients for KMT2C

In [19]:
all_patients = list(combined_maf['Patient_ID'].unique())

In [51]:
all_probs_KMT2C = []

for patient in all_patients: 

    RA_maf = coding_maf[coding_maf['Patient_ID']==patient]
    RA_maf_clones = list(RA_maf['Tumor_Sample_Barcode'].unique() )

    ## total number of mutations in the target gene N0
    N0 = len(KMT2C_all_muts)
    ## total number of clones
    N_c = len( coding_maf['Tumor_Sample_Barcode'].unique() )

    # total number of mutations in this patient
    M = len(RA_maf)
    # initialize a Pij matrix where you have a 1 if a clone was mutated 
    P_pat =np.zeros( (len(RA_maf_clones), 2 ))
    
    gene="KMT2C"
    
    clone_idx=0
    for clone in RA_maf_clones:
        clone_df = RA_maf[RA_maf['Tumor_Sample_Barcode'] == clone]
        mutations_in_clone = clone_df['Hugo_Symbol'].unique()
        Mij = len(list(mutations_in_clone))
        
        term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
        
        # probability that a given clone has no mutation in the gene
        P_pat[clone_idx, 0] = term_to_calc
        
        # probability that a given clone has one or more mutations in the gene
        P_pat[clone_idx, 1] = 1- term_to_calc
        clone_idx +=1

    P_prob_df = pd.DataFrame(P_pat, columns=['no_KMT2C', 'KMT2C'])
    P_prob_df.insert(0, 'clone', RA_maf_clones)
    all_probs_KMT2C.append(P_prob_df)
    

In [52]:
probs_1326 = all_probs_KMT2C[11]

## score if no KMT2C mutation in this patient

In [53]:
probs_1326_no_KMT2C = probs_1326['no_KMT2C'].to_list()
logs_no_KMT2C_1326 = [ np.log(p) for p in probs_1326_no_KMT2C]
print( -np.sum(logs_no_KMT2C_1326))

2.785239956066282


## real distribution

In [54]:
KMT2C_clones_RA1326= ['1326_CL2','1326_CL9']

In [55]:
P_1326_prob_df_KMT2C_clones = probs_1326[probs_1326['clone'].isin(KMT2C_clones_RA1326)]
P_1326_prob_df_non_KMT2C_clones = probs_1326[~probs_1326['clone'].isin(KMT2C_clones_RA1326)]

P_1326_prob_df_KMT2C_clones_list = P_1326_prob_df_KMT2C_clones['KMT2C'].to_list()
logs_real_KMT2C = [ -np.log(p) for p in P_1326_prob_df_KMT2C_clones_list]

P_1326_no_KMT2C_clones_list = P_1326_prob_df_non_KMT2C_clones['no_KMT2C'].to_list()
logs_real_no_KMT2C = [ -np.log(p) for p in P_1326_no_KMT2C_clones_list]


np.sum( np.concatenate((logs_real_KMT2C, logs_real_no_KMT2C)))

4.547794414031647

In [43]:
all_probs_KMT2C

[        clone  no_KMT2C     KMT2C
 0    1547_CL1  0.480690  0.519310
 1    1547_CL8  0.899154  0.100846
 2    1547_CL2  0.894124  0.105876
 3   1547_CL14  0.297655  0.702345
 4   1547_CL15  0.709490  0.290510
 5   1547_CL12  0.919544  0.080456
 6    1547_CL4  0.994434  0.005566
 7    1547_CL6  0.988898  0.011102
 8    1547_CL3  0.988898  0.011102
 9   1547_CL11  0.864503  0.135497
 10  1547_CL13  0.994434  0.005566,
        clone  no_KMT2C     KMT2C
 0   1598_CL2  0.931663  0.068337
 1   1598_CL1  0.867789  0.132211
 2   1598_CL8  0.700242  0.299758
 3  1598_CL10  0.269250  0.730750
 4   1598_CL5  0.883355  0.116645
 5   1598_CL4  0.931663  0.068337
 6   1598_CL7  0.793799  0.206201
 7   1598_CL9  0.867789  0.132211
 8   1598_CL3  0.899186  0.100814
 9   1598_CL6  0.793799  0.206201,
       clone  no_KMT2C     KMT2C
 0  1078_CL1  0.459814  0.540186
 1  1078_CL5  0.590613  0.409387
 2  1078_CL2  0.840239  0.159761
 3  1078_CL6  0.266166  0.733834
 4  1078_CL4  0.965897  0.034103,
     

In [28]:
all_probs_order = [ 1547, 1598, 1078, 1045, 1113, 1644, 2819, 2974, 1035,1534, 1558, 1326, 2389, 2542, 1002  ]
len(all_probs_order)

15

In [29]:
pats_with_no_KMT2C = [1598, 1078, 1045, 1113, 1644, 2819, 2974, 1035,1534,1002]
pats_with_no_KMT2C_idx =[1, 2, 3, 4, 5, 6, 7, 8, 9, 14]

In [44]:
probs_pats_no_KMT2C = []
probs_pats_no_KMT2C_scores = []
probs_pats_no_KMT2C_scores_dict = {}
for idx in pats_with_no_KMT2C_idx:
    score = 0
    probs_pats_no_KMT2C.append(all_probs_KMT2C[idx])
    pat_probs = all_probs_KMT2C[idx]
    pat_probs_no_KMT2C = pat_probs['no_KMT2C'].to_list()
    logs_no_KMT2C = [ np.log(p) for p in pat_probs_no_KMT2C]
    score = -np.sum(logs_no_KMT2C)
    
    probs_pats_no_KMT2C_scores.append(score)
    pat = all_probs_order[idx]
    probs_pats_no_KMT2C_scores_dict[pat] = score


In [45]:
probs_pats_no_KMT2C_scores_dict

{1598: 2.785767361195695,
 1078: 2.8359286696937476,
 1045: 2.787239669333773,
 1113: 2.743758971093486,
 1644: 2.762563428360604,
 2819: 2.760069539439102,
 2974: 2.676423530556352,
 1035: 2.7436121458661056,
 1534: 2.891962217736308,
 1002: 2.56378181686393}

In [46]:
pats_with_KMT2C_idx=[0, 10,11,12,13]

In [47]:
all_KMT2C_clones = list(KMT2C_all_muts['Tumor_Sample_Barcode'].unique())

In [56]:

probs_pats_with_KMT2C = []
probs_pats_KMT2C_scores = []
probs_pats_with_KMT2C_scores_dict = {}
for idx in pats_with_KMT2C_idx:
    #score=0
    running_score=[]
    pat_probs = all_probs_KMT2C[idx]
    all_clones_in_pat = list(pat_probs['clone'])
    for clone in all_clones_in_pat:
        pat_probs_clone = pat_probs[pat_probs['clone']==clone]
        if clone in all_KMT2C_clones:

            pat_probs_clone_KMT2C = pat_probs_clone['KMT2C'].to_list()
            logs_KMT2C = [ - np.log(p) for p in pat_probs_clone_KMT2C]
            running_score.extend(logs_KMT2C)
        else:
            pat_probs_no_KMT2C = pat_probs_clone['no_KMT2C'].to_list()
            logs_no_KMT2C = [ - np.log(p) for p in pat_probs_no_KMT2C]
            running_score.extend(logs_no_KMT2C)
            
    score = np.sum(running_score)
    
    probs_pats_with_KMT2C.append(pat_probs)
    probs_pats_KMT2C_scores.append(score)
    pat = all_probs_order[idx]
    probs_pats_with_KMT2C_scores_dict[pat] = score


In [57]:
probs_pats_with_KMT2C_scores_dict

{1547: 2.8031613827454733,
 1558: 3.403834833843852,
 1326: 4.547794414031647,
 2389: 14.714030365894862,
 2542: 8.820179860729803}

## test any gene

In [61]:
def calculate_distributions(gene):

    gene_all_muts = coding_maf[coding_maf['Hugo_Symbol']==gene]
    all_probs = []
    
    for patient in all_patients: 
    
        RA_maf = coding_maf[coding_maf['Patient_ID']==patient]
        RA_maf_clones = list(RA_maf['Tumor_Sample_Barcode'].unique() )
        
        ## total number of mutations N0
        N0 = len(gene_all_muts)
        ## total number of mutated clones N_c
        N_c = len( coding_maf['Tumor_Sample_Barcode'].unique() )
        
        # total number of mutations in this patient
        M = len(RA_maf)
        # initialize a Pij matrix where you have a 1 if a clone was mutated 
        P_pat =np.zeros( (len(RA_maf_clones), 2 ))
        
        clone_idx=0
        for clone in RA_maf_clones:
            clone_df = RA_maf[RA_maf['Tumor_Sample_Barcode'] == clone]
            mutations_in_clone = clone_df['Hugo_Symbol'].unique()
            Mij = len(list(mutations_in_clone))
            term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0

            # probability that a given clone has no mutation in the gene
            P_pat[clone_idx, 0] = term_to_calc
            
            
            # probability that a given clone has a mutation in the gene
            P_pat[clone_idx, 1] = 1-term_to_calc
            clone_idx +=1

    
        P_prob_df = pd.DataFrame(P_pat, columns=['no_gene_mutations', 'gene_mutation'])
        P_prob_df.insert(0, 'clone', RA_maf_clones)
        all_probs.append(P_prob_df)
    return all_probs

In [62]:
esr1_Pm = calculate_distributions('ESR1')

In [63]:
esr1_Pm

[        clone  no_gene_mutations  gene_mutation
 0    1547_CL1           0.848241       0.151759
 1    1547_CL8           0.976215       0.023785
 2    1547_CL2           0.974977       0.025023
 3   1547_CL14           0.762891       0.237109
 4   1547_CL15           0.925434       0.074566
 5   1547_CL12           0.981180       0.018820
 6    1547_CL4           0.998735       0.001265
 7    1547_CL6           0.997472       0.002528
 8    1547_CL3           0.997472       0.002528
 9   1547_CL11           0.967581       0.032419
 10  1547_CL13           0.998735       0.001265,
        clone  no_gene_mutations  gene_mutation
 0   1598_CL2           0.984091       0.015909
 1   1598_CL1           0.968411       0.031589
 2   1598_CL8           0.922709       0.077291
 3  1598_CL10           0.746270       0.253730
 4   1598_CL5           0.972310       0.027690
 5   1598_CL4           0.984091       0.015909
 6   1598_CL7           0.949127       0.050873
 7   1598_CL9           0.9